In [7]:
##do_runcode
##%overwritefile
##%file:src/dor_runcode.py
##%noruncode
    def dor_runcode(self,return_code,fil_ename,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):    
        ##runprg
        ##runprgargs
        return_code=return_code
        fil_ename=fil_ename
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        runprg=self.get_magicsbykey(magics,'runprg')
        runprgargs=self.get_magicsbykey(magics,'runprgargs')
        ##代码运行前
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,1)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        self._logln("The process :"+fil_ename)
        p = self.create_jupyter_subprocess([runprg]+ runprgargs,cwd=None,shell=False,env=self.addkey2dict(magics,'env'))
        self.g_rtsps[str(p.pid)]=p
        return_code=p.returncode
        ##代码启动后
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,2)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        
        if len(self.addkey2dict(magics,'showpid'))>0:
            self._write_to_stdout("The process PID:"+str(p.pid)+"\n")
        while p.poll() is None:
            p.write_contents(magics)
        self._write_to_stdout("The process end:"+str(p.pid)+"\n")
        # wait for threads to finish, so output is always shown
        p._stdout_thread.join()
        p._stderr_thread.join()
        # del self.g_rtsps[str(p.pid)]
        p.write_contents(magics)
        ##
        ##调用接口
        return_code=p.returncode
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,3,3)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        ##代码运行结束
        if p.returncode != 0:
            self._log("Executable exited with code {}".format(p.returncode),2)
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Runprg/src/dor_runcode.py created successfully


In [5]:
##do_create_codefile
##%overwritefile
##%file:src/dor_create_codefile.py
##%noruncode
    def dor_create_codefile(self,magics,code, silent, store_history=True,
                    user_expressions=None, allow_stdin=True):    
        ##runprg
        ##runprgargs
        return_code=0
        fil_ename=''
        bcancel_exec=False
        retinfo=self.get_retinfo()
        retstr=''
        ##调生成文件前接口
        bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,1)
        if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        with self.new_temp_file(suffix='.py',dir=os.path.abspath('')) as source_file:
            source_file.write(code)
            source_file.flush()
            newsrcfilename=source_file.name
            fil_ename=newsrcfilename
            return_code=True
            # Generate new src file
            ##调生成文件后接口
            bcancel_exec,retstr=self.raise_plugin(code,magics,return_code,fil_ename,1,2)
            if bcancel_exec:return bcancel_exec,retinfo,magics, code,fil_ename,retstr
            if len(self.addkey2dict(magics,'file'))>0:
                    fil_ename=magics['file'][0]
            else: fil_ename=source_file.name
        if len(self.addkey2dict(magics,'noruncode'))>0:
            bcancel_exec=True
            retinfo= self.get_retinfo()
            return bcancel_exec,retinfo,magics, code,fil_ename,retstr
        
        return bcancel_exec,retinfo,magics, code,fil_ename,retstr

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Runprg/src/dor_create_codefile.py created successfully


In [6]:
##do_preexecute
##%overwritefile
##%file:src/dor_preexecute.py
##%noruncode
    def dor_preexecute(self,code,magics,silent, store_history=True,
                user_expressions=None, allow_stdin=False):        
        ##runprg
        ##runprgargs
        bcancel_exec=False
        retinfo=self.get_retinfo()
        if len(self.addkey2dict(magics,'replcmdmode'))>0:
            bcancel_exec=True
            retinfo= self.send_replcmd(code, silent, store_history=store_history,
                user_expressions=user_expressions, allow_stdin=allow_stdin)
            return bcancel_exec,retinfo,magics, code
        if (len(self.addkey2dict(magics,'noruncode'))>0 
            and ( len(self.addkey2dict(magics,'command'))>0 
            or len(self.addkey2dict(magics,'pythoncmd'))>0)):
            bcancel_exec=True
        return bcancel_exec,retinfo,magics, code

[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/kernel/Runprg/src/dor_preexecute.py created successfully
